# Optimized data analysis

In [1]:
from __future__ import print_function
import numpy as np
#import matplotlib.pyplot as plt
import math
import scipy.interpolate
import os
#%matplotlib inline
#plt.rcParams['figure.figsize'] = (15, 8)

# only with Python2.7 !!
import sys
sys.path.insert(0,'/usr/lib/python2.7/pyobs-master/')
from pyobs import *

In [2]:
def round_on_error(value, error):
    significant_digits = 1-int(math.floor(math.log(error, 10)))
    return round(value,significant_digits)

def get_2_significant(value):
    return round(value,1-int(math.floor(math.log(value, 10))))

def get_2_significant_0(value):
    return int(round(10.0/(10.0**int(math.floor(math.log(value, 10))))*value,0))

def get_position_sign(error):
    return 1-int(math.floor(math.log(error, 10)))
   
# Function that given a value and an error returns value(error) with two significant digits.    
def printwe(value,error):
    e = get_2_significant(error)
    a = get_position_sign(e)
    e = get_2_significant_0(error)
    stri = '{:.' + str(a) + 'f}({})' 
    print(stri.format(value,e))

## Define I/O file paths

In [330]:
numobservables = 4

In [331]:
results_dir = 'results/stau2.txt'
#results_dir = 'results/newfile.txt'

In [338]:
data_directories = ['data/datahmc.txt',
                    'data/dataaltw.txt',
                    'data/dataalti.txt',
                    'data/dataaltw2.txt',
                    'data/dataaltw3.txt',
                    'data/dataaltw4.txt',
                    'data/dataaltw6.txt',
                    'data/datadoubaltw.txt'
                    ]

## Check if up to date

In [339]:
update = [True]*len(data_directories)

In [340]:
if(os.path.isfile(results_dir)):
    results = np.genfromtxt(results_dir, dtype=([('Data_directory', 'S30'), ('Stau', '<f8'),   ('N_configs', '<i4'), 
                ('t2E1', '<f8'), ('errt2E1', '<f8'), ('taut2E1', '<f8'), ('errtaut2E1', '<f8'),
                ('t2E5', '<f8'), ('errt2E5', '<f8'), ('taut2E5', '<f8'), ('errtaut2E5', '<f8'),
                ('t2E7', '<f8'), ('errt2E7', '<f8'), ('taut2E7', '<f8'), ('errtaut2E7', '<f8'),
                ('Q', '<f8'), ('errQ', '<f8'), ('tauQ', '<f8'), ('errtauQ', '<f8')]))
    for i in range(len(results)):
        nconfigs_dir = sum(1 for line in open(results['Data_directory'][i]))
        if( results['N_configs'][i] == nconfigs_dir ):
            update[i] = False

In [341]:
update

[False, False, False, False, False, False, True, False]

## Update if not up to date

In [342]:
results_data = []

for i in range(len(data_directories)):
    new_row = []
    if(update[i] == True):
        stau = 2.0
        nconfigs_dir = sum(1 for line in open(data_directories[i]))
        new_row.append(data_directories[i])
        new_row.append(stau)
        new_row.append(nconfigs_dir)
        
        for j in range(numobservables):
            observable = list(np.loadtxt(data_directories[i], usecols=j))
            MCtime = list(np.arange(1,len(observable)+1,1))

            corr = observa()
            einfo = errinfo()
            einfo.addEnsemble(0, Stau=stau)
            corr.primary_observable(0,'$Q$ HMC', [0], ['R0'], [MCtime], [observable], (1,1))
            [obs, eobs]= corr.vwerr(errinfo=einfo)
            [tauobs, etauobs] = corr.tauint(errinfo=einfo)
            tauobs = tauobs[0][0][0]
            etauobs = etauobs[0][0][0]

            new_row.append(obs)
            new_row.append(eobs)
            new_row.append(tauobs)
            new_row.append(etauobs)
        
        if(i == 0):
            results_data = np.array([new_row])
        else:
            results_data = np.append(results_data, [new_row], axis=0) 
    else:
        new_row.append(list(results[i]))
        
        if(i == 0):
            results_data = np.array(new_row)
        else:
            results_data = np.append(results_data, new_row, axis=0)
            

np.savetxt(results_dir, results_data, fmt='%s')
    
      

In [343]:
results = np.genfromtxt("results/check.txt", dtype=([('Data_directory', 'S30'), ('Stau', '<f8'),   ('N_configs', '<i4'), 
                ('t2E1', '<f8'), ('errt2E1', '<f8'), ('taut2E1', '<f8'), ('errtaut2E1', '<f8'),
                ('t2E5', '<f8'), ('errt2E5', '<f8'), ('taut2E5', '<f8'), ('errtaut2E5', '<f8'),
                ('t2E7', '<f8'), ('errt2E7', '<f8'), ('taut2E7', '<f8'), ('errtaut2E7', '<f8'),
                ('Q', '<f8'), ('errQ', '<f8'), ('tauQ', '<f8'), ('errtauQ', '<f8')]))

|Algorithm|Statistics| $t2E1$| $\tau_{\text{int}}$ | $\tau_{\text{int}}/\tau_{\text{int}}^{\text{(HMC)}}$
|:-:|:-:|:-:|:-:|:-:|
|HMC| {{results['N_configs'][0]}} | {{printwe(results['t2E1'][0],results['errt2E1'][0])}}| {{printwe(tauQhmc, etauQhmc)}} | - |
